In [109]:
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import re
import math
plt.style.use('fivethirtyeight')
import requests
from lxml import html
from lxml import etree

from bs4 import BeautifulSoup

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import openpyxl

In [110]:
#SAVING TEAMS HERE
caseySquad = ["Drew Brees", "Patrick Mahomes", "Michael Thomas", "Jared Cook", "Raheem Mostert", "Derrick Henry", "Mark Ingram", "Tyreek Hill", "Damien Williams", "Wil Lutz", "Harrison Butker", "SF", "KC"]
kellySquad = ["Lamar Jackson", "Drew Brees", "Mark Ingram", "Mark Andrews", "Raheem Mostert", "Deebo Samuel", "Jared Cook", "Alvin Kamara", "George Kittle", "Wil Lutz", "Robbie Gould", "NE", "BAL"]
johnMSquad = ["Lamar Jackson", "Drew Brees", "Michael Thomas", "Alvin Kamara", "Jared Cook", "Mark Ingram", "Mark Andrews", "Marquise Brown", "Tyreek Hill", "Wil Lutz", "Justin Tucker", "BAL", "NO"]
timSquad = ["Lamar Jackson", "Drew Brees", "Michael Thomas", "Jared Cook", "Mark Ingram", "Julian Edelman", "Sony Michel", "Tyler Lockett", "Aaron Jones", "Wil Lutz", "Justin Tucker", "BAL", "NE"]
skipSquad = ["Drew Brees", "Deshaun Watson", "Tyler Lockett", "Sony Michel", "Julian Edelman", "Michael Thomas", "Alvin Kamara", "Deandre Hopkins", "Carlos Hyde", "Nick Folk", "Wil Lutz", "NO", "NE"]
micahSquad = ["Lamar Jackson", "Drew Brees", "George Kittle", "Michael Thomas", "Davante Adams", "Raheem Mostert", "Travis Kelce", "Mark Ingram", "Deandre Hopkins", "Justin Tucker", "Robbie Gould", "BAL", "SF"]
troySquad = ["Lamar Jackson", "Drew Brees", "Alvin Kamara", "Michael Thomas", "Taysom Hill", "Tyreek Hill", "Mark Andrews", "Mark Ingram", "Aaron Jones", "Justin Tucker", "Wil Lutz", "BAL", "NO"]
daveBSquad = ["Lamar Jackson", "Drew Brees", "Raheem Mostert", "George Kittle", "Mark Ingram", "Mark Andrews", "Jared Cook", "Michael Thomas", "Tyreek Hill", "Justin Tucker", "Robbie Gould", "SF", "BAL"]
warnerSquad = ["Lamar Jackson", "Drew Brees", "Michael Thomas", "Alvin Kamara", "Mark Ingram", "Marquise Brown", "George Kittle", "Emmanuel Sanders", "Tyreek Hill", "Justin Tucker", "Wil Lutz", "SF", "BAL"]
jackSquad = ["Drew Brees", "Patrick Mahomes", "Alvin Kamara", "Michael Thomas", "Travis Kelce", "George Kittle", "Deebo Samuel", "Mark Andrews", "Julian Edelman", "Wil Lutz", "Robbie Gould", "SF", "BAL"]
johnRSquad = ["Drew Brees", "Patrick Mahomes", "Travis Kelce", "Michael Thomas", "Alvin Kamara", "Aaron Jones", "George Kittle", "Mark Ingram", "Tyreek Hill", "Wil Lutz", "Harrison Butker", "NO", "SF"]
johnSSquad = ["Lamar Jackson", "Drew Brees", "Michael Thomas", "George Kittle", "Alvin Kamara", "Raheem Mostert", "Mark Ingram", "Tyreek Hill", "Mark Andrews", "Wil Lutz", "Justin Tucker", "SF", "BAL"]
daveGSquad = ["Drew Brees", "Patrick Mahomes", "George Kittle", "Michael Thomas", "Alvin Kamara", "Travis Kelce", "Damien Williams", "Sony Michel", "Raheem Mostert", "Wil Lutz", "Justin Tucker", "NO", "KC"]
philSquad = ["Lamar Jackson", "Patrick Mahomes", "Michael Thomas", "Davante Adams", "Mark Ingram", "Travis Kelce", "George Kittle", "Dalvin Cook", "Tyler Lockett", "Justin Tucker", "Wil Lutz", "BAL", "NO"]
juliusSquad = ["Lamar Jackson", "Drew Brees", "Alvin Kamara", "Michael Thomas", "James White", "Tyreek Hill", "Aaron Jones", "Mark Andrews", "Tyler Lockett", "Wil Lutz", "Justin Tucker", "BAL", "NE"]


In [111]:
def get_scoring_data(input_url):
    """This function returns all the scoring plays from a given game as a list of strings"""
    """this format is for the ESPN Gamecast version of games"""
    
    assert type(input_url) == str , "input_url must be a string -- try putting it in quotes!"
    
    url = input_url
    res = requests.get(url)
    tree = html.fromstring(res.content)
    scoring = tree.xpath('//div[@class="headline"]/text()')
    as_string = [str(i)for i in scoring]
    return as_string

In [112]:
def scoring_by_player(team1_at_team2_string):
    new = []
    pos = 0
    for i in team1_at_team2_string:
        if "(Two-Point Pass Conversion Failed)" in i:
            new.append((str(i.rpartition("(Two-Point Pass Conversion Failed)")[0])))
        elif '(' in i:
            new.extend((i.split("(")))
        else:
            new.append(i)
    new = [i.strip() for i in new]
    new = [i.strip(')') for i in new]


    newer = []
    pos = 0
    for i in new:
        if 'pass' in i:
            add = i.split('pass from')
            newer.extend(add)
            newer[pos + 1] = newer[pos + 1] + ' ' + newer[pos][-6:] + 'pass'
            newer[pos] = newer[pos] + 'pass'
            pos += 2 
        elif "Pass" in i:
            add = i.split("Pass")
            newer.extend(add)
            newer[pos] = newer[pos] + "Two-Point Conversion"

        else:
            newer.append(i)
            pos += 1
    

    newer = [i.strip() for i in newer]
    return newer


In [113]:
def d_p_pt(scoring_by_player_func):
    import re
    import math
    regex1 = re.compile(r"(\w*? \w*? *?) [0-9|Kick|Pass|Run|for|Two|Safety]")
    regex2 = re.compile(r'\d{1,2}')
    regex3 = re.compile(r'\w+ \w+(.*)')
    
    scoring = scoring_by_player_func

    player = [re.findall(regex1, str(i))[0].strip() for i in scoring]
    dist = [re.findall(regex2, str(i)) for i in scoring]
    
    play_type = [re.findall(regex3,  str(i))[0].strip().lower() for i in scoring]
    
    for i in dist:
        if i == []:
            i.append(0)
    distances = [] 
    distances = [i[0] for i in dist]
    
    return distances, player, play_type

In [114]:
def make_game_df(d_p_pt_func):

    distances = d_p_pt_func[0]
    player = d_p_pt_func[1]
    play_type = d_p_pt_func[2]

    scores = pd.DataFrame(data = {'distance': pd.to_numeric(distances), 'player': player, 'play type': play_type})

    

    scores["FG"] = 0
    scores["TD"] = 0
    scores["Safety"] = 0
    scores["PAT"] = 0
    scores["PAT2"] = 0
    scores.loc[scores["play type"].str.contains('field goal'), "FG"] = 1 
    scores.loc[scores["play type"].str.contains("kick"), "PAT"] = 1
    scores.loc[scores["play type"].str.contains("two-point"), "PAT2"] = 1
    scores.loc[scores["play type"].str.contains("safety"), "Safety"] = 1
    scores.loc[scores["play type"].str.contains("pass"), "TD"] = 1
    scores.loc[scores["play type"].str.contains("run"), "TD"] = 1
    scores.loc[scores["play type"].str.contains("return"), "TD"] = 1
    scores.loc[scores["TD"] == scores["PAT2"], "TD"] = 0

    scores["points"] = (((scores["FG"] * scores['distance']) // 10) + 
                        ((5 * scores["TD"] + (scores["distance"]) // 10) * scores["TD"]) + 
                        (2 * scores["Safety"]) + 
                        (1 * scores["PAT"]) + 
                        (2 * scores["PAT2"]))

    scores.loc[(scores["distance"] > 55) & (scores["FG"] == 1), "points"] = scores["points"] + (scores["distance"] - 55)
    
    return scores

In [115]:
def get_single_game_points(input_url):
    """This combines all the functions written above into 1"""
    """Make sure the input_url format is for the ESPN Gamecast version of games in string form"""
    
    team1_at_team2_string = get_scoring_data(input_url)
    
    scoring_by_player_func = scoring_by_player(team1_at_team2_string)
    
    #if input_url == "https://www.espn.com/nfl/game/_/gameId/401131042":
    #    scoring_by_player_func = scoring_by_player_func[0:-2]
    
    d_p_pt_func = d_p_pt(scoring_by_player_func)
    
    game_df = make_game_df(d_p_pt_func)
    
    full_game = game_df.replace({"kick": "PAT"})
    
    return full_game

In [116]:
"""WILD CARD WEEKEND GAMES"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
titans_at_pats = get_single_game_points('https://www.espn.com/nfl/game?gameId=401131036').replace({"Logan Ryan": "TEN"})
bills_at_texans = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401131037').replace({"imi Fairbairn": "Ka'imi Fairbairn"})
vikes_at_saints = get_single_game_points("https://www.espn.com/nfl/game/_/gameId/401131038")
hawks_at_eagles = get_single_game_points("https://www.espn.com/nfl/game/_/gameId/401131039")

hawks_at_eagles

,distance,player,play type,FG,TD,Safety,PAT,PAT2,points
0,49,Jason Myers,49 yd field goal,1,0,0,0,0,4
1,46,Jake Elliott,46 yd field goal,1,0,0,0,0,4
2,5,Marshawn Lynch,5 yd run,0,1,0,0,0,5
3,0,Jason Myers,PAT,0,0,0,1,0,1
4,26,Jake Elliott,26 yd field goal,1,0,0,0,0,2
5,53,DK Metcalf,53 yd pass,0,1,0,0,0,10
6,53,Russell Wilson,53 yd pass,0,1,0,0,0,10
7,0,Jason Myers,PAT,0,0,0,1,0,1
8,38,Jake Elliott,38 yd field goal,1,0,0,0,0,3


In [117]:
"""WILD CARD EXCEL EXPORTER"""
###This won't run from Google Colab (Written using Jupyter Notebook, but copied here for sharing purposes)
###It doesn't change the data, just puts it in an excel readable format.

writer = pd.ExcelWriter("exportdoc.xlsx", engine='xlsxwriter')
vikes_at_saints.to_excel(writer, sheet_name = "v@s")
hawks_at_eagles.to_excel(writer, sheet_name = "h@e")
bills_at_texans.to_excel(writer, sheet_name = "b@t")
titans_at_pats.to_excel(writer, sheet_name = "t@p")

all_wc_scores = pd.concat([titans_at_pats, hawks_at_eagles, vikes_at_saints, bills_at_texans])[["player", "points"]]
wc = all_wc_scores.groupby("player").sum().reset_index() 
wc.to_excel(writer, sheet_name = "all wc scoring")

In [118]:
"""DIVISIONAL WEEKEND GAMES"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
vikes_at_niners = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401131040')
titans_at_ravens = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401131041')
texans_at_chiefs = get_single_game_points("https://www.espn.com/nfl/game/_/gameId/401131043").replace({"imi Fairbairn": "Ka'imi Fairbairn", "10 Yd" : "HOU", "jr. 10 yd return of blocked punt": "Blocked Punt "})
hawks_at_packers = get_single_game_points("https://www.espn.com/nfl/game/_/gameId/401131042")

vikes_at_niners

,distance,player,play type,FG,TD,Safety,PAT,PAT2,points
0,3,Kendrick Bourne,3 yd pass,0,1,0,0,0,5
1,3,Jimmy Garoppolo,3 yd pass,0,1,0,0,0,5
2,0,Robbie Gould,PAT,0,0,0,1,0,1
3,41,Stefon Diggs,41 yd pass,0,1,0,0,0,9
4,41,Kirk Cousins,41 yd pass,0,1,0,0,0,9
5,0,Dan Bailey,PAT,0,0,0,1,0,1
6,1,Tevin Coleman,1 yd run,0,1,0,0,0,5
7,0,Robbie Gould,PAT,0,0,0,1,0,1
8,39,Dan Bailey,39 yd field goal,1,0,0,0,0,3
9,35,Robbie Gould,35 yd field goal,1,0,0,0,0,3


In [119]:
"""DIVISIONAL EXCEL EXPORT"""
###Same as above...This won't run from Google Colab (Written using Jupyter Notebook, but copied here for sharing purposes)
###It doesn't change the data, just puts it in an excel readable format.

writer_div = pd.ExcelWriter("divisional.xlsx", engine='xlsxwriter')
vikes_at_niners.to_excel(writer_div, sheet_name = "v@n")
titans_at_ravens.to_excel(writer_div, sheet_name = "t@r")
texans_at_chiefs.to_excel(writer_div, sheet_name = "t@c")
hawks_at_packers.to_excel(writer_div, sheet_name = "h@p")


all_div_scores = pd.concat([vikes_at_niners, titans_at_ravens, texans_at_chiefs, hawks_at_packers])[["player", "points"]]
div = all_div_scores.groupby("player").sum().sort_values("points", ascending = False).reset_index()
div.to_excel(writer_div, sheet_name = "div scoring")

In [120]:
"""CHAMPIONSHIP GAMES"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
packers_at_niners = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401131045')
titans_at_chiefs = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401131044')


packers_at_niners

,distance,player,play type,FG,TD,Safety,PAT,PAT2,points
0,36,Raheem Mostert,36 yd run,0,1,0,0,0,8
1,0,Robbie Gould,PAT,0,0,0,1,0,1
2,54,Robbie Gould,54 yd field goal,1,0,0,0,0,5
3,9,Raheem Mostert,9 yd run,0,1,0,0,0,5
4,0,Robbie Gould,PAT,0,0,0,1,0,1
5,27,Robbie Gould,27 yd field goal,1,0,0,0,0,2
6,18,Raheem Mostert,18 yd run,0,1,0,0,0,6
7,0,Robbie Gould,PAT,0,0,0,1,0,1
8,9,Aaron Jones,9 yd pass,0,1,0,0,0,5
9,9,Aaron Rodgers,9 yd pass,0,1,0,0,0,5


In [121]:
"""CHAMPIONSHIPS EXCEL EXPORT"""
###Same as above...This won't run from Google Colab (Written using Jupyter Notebook, but copied here for sharing purposes)
###It doesn't change the data, just puts it in an excel readable format.



writer_ship = pd.ExcelWriter("ship.xlsx", engine='xlsxwriter')
packers_at_niners.to_excel(writer_ship, sheet_name = "p@n")
titans_at_chiefs.to_excel(writer_ship, sheet_name = "t@c")

all_ship_scores = pd.concat([packers_at_niners, titans_at_chiefs])[["player", "points"]]
ship = all_ship_scores.groupby("player").sum().sort_values("points", ascending = False).reset_index()
ship.to_excel(writer_ship, sheet_name = "ship scoring")

In [122]:
"""SUPER BOWL"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
chiefs_v_niners = get_single_game_points('https://www.espn.com/nfl/game?gameId=401131047')


chiefs_v_niners

,distance,player,play type,FG,TD,Safety,PAT,PAT2,points
0,38,Robbie Gould,38 yd field goal,1,0,0,0,0,3
1,1,Patrick Mahomes,1 yd run,0,1,0,0,0,5
2,0,Harrison Butker,PAT,0,0,0,1,0,1
3,31,Harrison Butker,31 yd field goal,1,0,0,0,0,3
4,15,Kyle Juszczyk,15 yd pass,0,1,0,0,0,6
5,15,Jimmy Garoppolo,15 yd pass,0,1,0,0,0,6
6,0,Robbie Gould,PAT,0,0,0,1,0,1
7,42,Robbie Gould,42 yd field goal,1,0,0,0,0,4
8,1,Raheem Mostert,1 yd run,0,1,0,0,0,5
9,0,Robbie Gould,PAT,0,0,0,1,0,1


In [123]:
"""SUPER BOWL EXCEL EXPORT"""
###Same as above...This won't run from Google Colab (Written using Jupyter Notebook, but copied here for sharing purposes)
###It doesn't change the data, just puts it in an excel readable format.

writer_sb = pd.ExcelWriter("sb.xlsx", engine='xlsxwriter')
chiefs_v_niners.to_excel(writer_sb, sheet_name = "Super Bowl")

all_sb_scores = pd.concat([chiefs_v_niners])[["player", "points"]]
sb = all_sb_scores.groupby("player").sum().sort_values("points", ascending = False).reset_index()
sb.to_excel(writer_sb, sheet_name = "Super Bowl scoring")

In [124]:
def make_team(squad):
    team = pd.DataFrame(squad)
    #Adding Wild Card results
    team = team.merge(wc, how = "left", left_on = 0, right_on = "player")
    team = team.drop(columns = "player").rename(columns = {0: "player"}).fillna(0)
    
    team = team.merge(div, how = "left", left_on = "player" , right_on = "player").fillna(0)
    team = team.rename(columns = {"points_x": "WC", "points_y": "Div"})
    
    team = team.merge(ship, how = 'left', left_on = "player", right_on = "player").fillna(0)
    team = team.rename(columns = {"points": "Ship"})
    
    team = team.merge(sb, how = 'left', left_on = "player", right_on = "player").fillna(0)
    team = team.rename(columns = {"points": "SB"})
    
    team["Total"] = team["WC"] + team["Div"] + team["Ship"] + team["SB"]
    team = team.append(team.sum().rename('Total'))
    return team

In [125]:
"""COMBINING PLAYERS' SCORING DATA AND PARTICIPANTS' TEAMS"""

casey = make_team(caseySquad)
kelly = make_team(kellySquad)
johnM = make_team(johnMSquad)
tim = make_team(timSquad)
skip = make_team(skipSquad)
micah = make_team(micahSquad)
julius = make_team(juliusSquad)
troy = make_team(troySquad)
daveB = make_team(daveBSquad)
warner = make_team(warnerSquad)
jack = make_team(jackSquad)
johnR = make_team(johnRSquad)
johnS = make_team(johnSSquad)
daveG = make_team(daveGSquad)
phil = make_team(philSquad)

jack

,player,WC,Div,Ship,SB,Total
0,Drew Brees,7.0,0.0,0.0,0.0,7.0
1,Patrick Mahomes,0.0,26.0,30.0,15.0,71.0
2,Alvin Kamara,5.0,0.0,0.0,0.0,5.0
3,Michael Thomas,0.0,0.0,0.0,0.0,0.0
4,Travis Kelce,0.0,15.0,0.0,5.0,20.0
5,George Kittle,0.0,0.0,0.0,0.0,0.0
6,Deebo Samuel,0.0,0.0,0.0,0.0,0.0
7,Mark Andrews,0.0,0.0,0.0,0.0,0.0
8,Julian Edelman,5.0,0.0,0.0,0.0,5.0
9,Wil Lutz,8.0,0.0,0.0,0.0,8.0


In [126]:
"""FINAL TEAMS & POINTS EXCEL EXPORT"""
###Again...This won't run from Google Colab (Written using Jupyter Notebook, but copied here for sharing purposes)
###It doesn't change the data, just puts it in an excel readable format.

##Eventually I should make this into a function

casey.to_excel(writer_sb, sheet_name = "casey")
kelly.to_excel(writer_sb, sheet_name = "kelly")
johnM.to_excel(writer_sb, sheet_name = "johnM")
tim.to_excel(writer_sb, sheet_name = "tim")
skip.to_excel(writer_sb, sheet_name = "skip")
micah.to_excel(writer_sb, sheet_name = "micah")
julius.to_excel(writer_sb, sheet_name = "julius")
troy.to_excel(writer_sb, sheet_name = "troy")
daveB.to_excel(writer_sb, sheet_name = "daveB")
warner.to_excel(writer_sb, sheet_name = "warner")
jack.to_excel(writer_sb, sheet_name = "jack")
johnR.to_excel(writer_sb, sheet_name = "johnR")
johnS.to_excel(writer_sb, sheet_name = "johnS")
daveG.to_excel(writer_sb, sheet_name = "daveG")
phil.to_excel(writer_sb, sheet_name = "phil")

writer_sb.save()